In [ ]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr

delta_list=['50','45', '40', '35', '30', '25','20','15','10']#,'5','55','60','65','70','75','80','85']
from tqdm.notebook import tqdm
list1=[100,125,150,175,200]
N=[1]#,0,1,2,3,4]
import warnings
warnings.filterwarnings("ignore")
leverage=6
def closest_strike(group, input_value):
    return     (np.abs(group['OPT_Close'] - input_value)).argmin()

direc=r'C:\Projects\Data\BN\2023\Monthly/'
exp_df = pd.read_csv(r'C:\Projects\Calendar 11-12\data\BN weekly 2016-2022/WeeklyExpiry.csv',parse_dates = ["Weekly_Expiry_Date","date"],dayfirst =True,usecols = ["Weekly_Expiry_Date","date"])
monthly_exp=pd.read_csv(r'C:\Projects\Straddles\NEW/MonthlyExpiry_2023.csv',parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date"])
weekly_exp=pd.read_csv(r'C:\Projects\Straddles\NEW/WeeklyExpiry_2023.csv',parse_dates = ["Weekly_Expiry_Date","date"],dayfirst =True,usecols = ["Weekly_Expiry_Date","date"])
BN_data=pd.read_csv(r'C:\Projects\Data\BN\2023\Monthly/without contract filter//BANKNIFTY-I.csv',dayfirst=True,parse_dates=['Date'])
BN_data['Date']=pd.to_datetime(BN_data['Date'])
BN_data_CE=BN_data[BN_data["Option_Type"] == "CE"]
BN_data_PE=BN_data[BN_data["Option_Type"] == "PE"]

for delta in tqdm(delta_list):
    print(delta)
    idx = pd.read_csv(direc + "/BANKNIFTY-I.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])

    idx2 = pd.read_csv( r'C:\Projects\Data\BN\2023\Monthly\without contract filter' + "/BANKNIFTY-II.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])
    idx=idx[idx['Date']>=pd.to_datetime('27-05-2016')].reset_index(drop=True)
    idx=idx[idx['Date']<=pd.to_datetime('20-01-2023')].reset_index(drop=True)
    idx2=idx2[idx2['Date']>=pd.to_datetime('27-05-2016')].reset_index(drop=True)
    idx2=idx2[idx2['Date']<=pd.to_datetime('20-01-2023')].reset_index(drop=True)
    for nnn in N:
            print(nnn)
            
            output_path=r'C:\Projects\Straddles\NEW\2016-2023/M2 rollover-Monthly Base Case'+'/'+delta+'_delta'+'/'+str(nnn)
            # print(output_path)
            if not os.path.exists(output_path):
                os.makedirs(output_path)

            # xxx
            

            idx_ce = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
            idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")
            idx2_ce = idx2[ (idx2["Option_Type"] == "CE")]
            idx2_pe = idx2[ (idx2["Option_Type"] == "PE")]
            final=idx_ce.merge(idx_pe,left_on='Date_CE',right_on='Date_PE')
            final = pd.merge(final, monthly_exp, left_on = 'Date_CE', right_on='Date',how = 'left')
            final=pd.merge(final, weekly_exp, left_on = 'Date_CE', right_on='date',how = 'left')

            final = final.T.drop_duplicates().T  #Drop Duplicates Columns
            if 'Strike_PE' not in final.columns:
                    final['Strike_PE']=final['Strike_CE']
                    print(delta)
            final.rename(columns = {'Date_CE':'Date'}, inplace = True)

            final.to_csv(output_path+"/check-1data-w3.csv")


            final=pd.read_csv(output_path+"/check-1data-w3.csv",parse_dates=['Date','curr_exp_date'],dayfirst=True)

            df=pd.read_csv(output_path+"/check-1data-w3.csv")
            status=1
            nnn1=nnn
            print(nnn1)
            if nnn>0:
                nnn1=nnn-1
            for idi,row in df.iterrows():
                if nnn1!=4:
                    if idi==0:
                        if nnn!=0:
                            df.loc[idi:idi+nnn1-1,'Status']=0
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,'status']=1
                        else:
                            df.loc[idi,"Marker"]='F'
                            df.loc[idi,'status']=1
                    elif (df.loc[idi-1,'Date'] == df.loc[idi-1,'curr_exp_date']):
                        # df.loc[idi,"Marker"]='M'
                        if nnn!=0:
                            # print(idi)
                            df.loc[idi:idi+nnn-1,"Status"]=0
                        # df.loc[idi+nnn1,"Marker"]='F'
                        df.loc[idi+nnn1,"status"]=1
                        # df.loc[idi+nnn-1,"status"]=1
                        df.loc[idi+nnn1,"Marker2"]='F'

                        
                    if (df.loc[idi,'curr_exp_date'] != df.loc[idi,'Weekly_Expiry_Date']):
                        if (df.loc[idi+1,'curr_exp_date'] == df.loc[idi+1,'Weekly_Expiry_Date']):
                            df.loc[idi,"Marker"]='L'
                            df.loc[idi+1,"Marker"]='F'
                    if (df.loc[idi,'curr_exp_date'] == df.loc[idi,'Weekly_Expiry_Date']):
                        df.loc[idi,"Marker3"]='F'
                        if df.loc[idi-1,"Marker3"]!='F':
                            df.loc[idi-1,"Marker3"]='L'
                        
                        
                elif nnn1==4:
                    if idi==0:
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,'status']=1
                    elif (df.loc[idi,'Date'] == df.loc[idi,'curr_exp_date']):
                        df.loc[idi-1,"Marker"]='F'
                        df.loc[idi,"Marker"]='L'
                        df.loc[idi-1:idi,'status']=1
                    if df.loc[idi,'status']!=1:
                        df.loc[idi,'Status']=0
            
            df.reset_index(drop=True,inplace=True)    
            df.to_csv(output_path+'/dte_test3-3-3-1.csv')
            
            for idi,row in df.iterrows():
                if nnn1!=0 and nnn!=4:
                    if df.loc[idi,'Status']==0:
                        df.drop(idi, axis=0, inplace=True)
                if nnn1==4:
                    if df.loc[idi,'status']!=1:
                        
                        df.drop(idi, axis=0, inplace=True)

            df.to_csv(output_path+'/dte_test3-3-3.csv')
            # xxx
            from tqdm.notebook import tqdm
            final=pd.DataFrame()
            dfg=df.groupby(['curr_exp_date'])
            len(dfg)
            temp_1=0
            temp=0
            for name,group in tqdm(dfg):
                    if temp==0:
                        strike_ce=group["Strike_CE"].iat[0]
                        date=group["Date"].iat[0]
                        strike_pe=group["Strike_PE"].iat[0]

                    if temp==1:
                        strike_ce=strike_new_ce
                        date=group["Date"].iat[0]
                        strike_pe=strike_new_pe


                    x1=x2=x3=y1=y2=y3=0

                    temp1=0
                    temp_x=0
                    for idi,row in group.iterrows():
                        # if temp==0:
                            date=group.loc[idi,'Date']
                            x_ce=idx[(idx['Date']==date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==strike_ce)]
                            x_pe=idx[(idx['Date']==date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==strike_pe)]
                            if nnn==0:
                                y_ce=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "CE") & (idx2["Strike"]==strike_ce_w2)]
                                y_pe=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "PE") & (idx2["Strike"]==strike_pe_w2)]

                            if len(x_ce):
                                group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                            if len(x_ce)==0:
                                    bndatace=BN_data_CE[(BN_data_CE['Date']==date) & (BN_data_CE['Strike']==strike_ce)]
                                    if len(bndatace):
                                        group.loc[idi,'Strike_CE']=x1=bndatace['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2=bndatace['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3=bndatace['Ticker'].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]

                            if len(x_pe):
                                
                                group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                            if len(x_pe)==0:
                                    bndatape=BN_data_PE[(BN_data_PE['Date']==date) & (BN_data_PE['Strike']==strike_pe) ]
                                    if len(bndatape):
                                        group.loc[idi,'Strike_PE']=y1=bndatape['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2=bndatape['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3=bndatape['Ticker'].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                            if row['Marker3']=='F':
                                    if temp_x==0:
                                        strike_ce_w2=strike_new_ce=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "CE") & (idx2["Strike"]==idx2["Delta_"+delta+"_Strike"])].reset_index(drop=True)['Strike'].iat[0]
                                        strike_pe_w2=strike_new_pe=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "PE") & (idx2["Strike"]==idx2["Delta_"+delta+"_Strike"])].reset_index(drop=True)['Strike'].iat[0]
                                        temp_x=1
                                    y_ce=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "CE") & (idx2["Strike"]==strike_ce_w2)]
                                    y_pe=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "PE") & (idx2["Strike"]==strike_pe_w2)]
                                    
                                    if len(y_ce):
                                        group.loc[idi,'Strike_CE']=yy1=y_ce["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=yy2=y_ce["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_CE']=yy3=y_ce["Ticker"].iat[0]
                                        group.loc[idi,'IV_CE']=y_ce["IV"].iat[0]
                                    if len(y_ce)==0:
                                        group.loc[idi,'Strike_CE']=yy1#x["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=yy2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_CE']=yy3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]
                                    if len(y_pe):
                                        group.loc[idi,'Strike_PE']=zz1=y_pe["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=zz2=y_pe["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_PE']=zz3=y_pe["Ticker"].iat[0]
                                        group.loc[idi,'IV_PE']=y_pe["IV"].iat[0]
                                    if len(y_pe)==0:
                                        group.loc[idi,'Strike_PE']=zz1#"Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=zz2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_PE']=zz3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                                    
                                    
                                    
                                    if temp==0:
                                        temp=1
                                        continue



                    final=final.append(group,ignore_index=True)

            final.to_csv(output_path+'/t3-w3-3.csv')
            # xxx
            df=pd.read_csv(output_path+'/t3-w3-3.csv')
            poe=0.01
            df['lotsize']=1
            df['qty']=1
            plvalue = []
            pl = 0
            transaction_cost = 0
            contract_value=0

            df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
            df['Margin']=df['Exposure']/leverage


            initial_equity=10000000
            equity=poe*initial_equity

            df.to_csv(output_path+"/unit check1=w3.csv")


            df=pd.read_csv(output_path+"/unit check1=w3.csv")
            # print(6,nnn1)
            temp=0

            for idi, row in df.iterrows():

                if idi==0:
                    df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                    pl=pl+df.loc[idi,'Points']
                # if row["Marker"] == "F":
                    df.loc[idi, 'pl_check'] = (pl+initial_equity)
                    df.loc[idi+1:, 'pl_check'] = np.nan

                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                    df.loc[idi+1: , 'units_new'] = np.nan

                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                    df.loc[idi+1:, 'qty_final'] = np.nan
                    df['qty_final']=df['qty_final'].ffill()
                    # df.loc[idi, 'plvalue_' + index] = 0
                    # df.loc[idi, 'transaction_cost'] = 0
                    contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                    continue

                else:
                    if nnn>0:
                        if row["Marker"] == "F":
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            contract_value= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])


                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl+initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan

                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                            pl=pl+df.loc[idi,'Points_final']
                        else:
                            df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                            pl=pl+df.loc[idi,'Points_final']

                        if row['Marker']=='L' :
                            contract_value+= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            df.loc[idi,'transaction_cost']=contract_value*0.01
                            pl=pl-df.loc[idi,'transaction_cost']
                            df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']-df.loc[idi,'transaction_cost']




            df['Points_final_2']=df['Points']*df['qty_final']
            df['Total_points']=df['Points_final'].cumsum()
            # print(7,nnn1)
            df.to_csv(output_path+'/pl_points_'+str(delta)+'.csv')

            df=pd.read_csv(output_path+'/pl_points_'+str(delta)+'.csv')
            temp=0
            df['Total_points']=df['Points_final'].cumsum()
            df['eq_curve']=df['Points_final']
            df.loc[0,'eq_curve']=initial_equity
            df['eq_curve']=df['eq_curve'].cumsum()
            df.to_csv(output_path+'/eq_curve_'+str(delta)+'.csv')
            print((output_path+'/eq_curve_'+str(delta)+'.csv'))
            print('Done')




